# 作业内容：

> 1，设计并训练深度神经网络，对夸克胶子喷注进行分类

# 参考答案


In [1]:
import numpy as np
from tqdm import tqdm
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os

In [2]:
#load data
with np.load('./data/QG_jets.npz') as f:
    raw_data = f['X']
    label = f['y']

In [3]:
#check and know the data
print('data tensor has such shape: {}'.format(raw_data.shape))
print('label tensor has such shape: {}'.format(label.shape))
print('head 5 lines for event No.1: \n{}'.format(raw_data[0, :5, :]))

data tensor has such shape: (100000, 139, 4)
label tensor has such shape: (100000,)
head 5 lines for event No.1: 
[[ 2.68769142e-01  3.56903171e-01  4.74138734e+00  2.20000000e+01]
 [ 1.60076377e-01 -2.55609533e-01  4.55022910e+00  2.20000000e+01]
 [ 1.14868731e+00 -6.24380156e-02  4.50385377e+00 -2.11000000e+02]
 [ 4.13159146e+00  1.73686350e-01  4.76622410e+00 -3.21000000e+02]
 [ 1.69599701e+00 -2.12177764e-01  4.79687162e+00 -2.11000000e+02]]


In [4]:
#as we already know that a row for an event has such variables: [pt, y, phi, pid]
#and our network wants a histogram of y and phi

#define the transformation function
def transform(data, bins=[28, 28], range_=[[-1.7, 1.7], [0, 2*np.pi]]):
    r'''
    Convert the raw data of 139 rows and 4 columns into a 28-by-28 2D heatmap.
    '''
    y = data[:, 1]
    phi = data[:, 2]
    res, _, _ = np.histogram2d(y, phi, bins=bins, range=range_, density=True)
    return res

#convert the data set
data = np.array([transform(item) for item in raw_data])
#do a simple scale
data /= data.max()

In [5]:
#check and save our pre-processed data
print('Pre-processed data tensor has such shape: {}'.format(data.shape))
if not os.path.exists('data'):
    os.mkdir('data')
np.save('./data/QG_proc_data.npy', data)
np.save('./data/QG_proc_label.npy', label)

Pre-processed data tensor has such shape: (100000, 28, 28)


In [6]:
#I'll skip the former step for saving time, so load the pre-processed data directly
data = np.load('./data/QG_proc_data.npy')
label = np.load('./data/QG_proc_label.npy')

#prepare the training and test set
data = np.expand_dims(data, -1)#keras need one more dimension (for channel)
label = keras.utils.to_categorical(label, 2)#one-hot trick
data_train, data_test, label_train, label_test = train_test_split(data, label, test_size=0.1)

print('train set: \ndata: {}, label: {}'.format(data_train.shape, label_train.shape))
print('test set: \ndata: {}, label: {}'.format(data_test.shape, label_test.shape))

train set: 
data: (90000, 28, 28, 1), label: (90000, 2)
test set: 
data: (10000, 28, 28, 1), label: (10000, 2)


In [7]:
#construct the model
num_classes = 2
input_shape = (28, 28, 1)
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax'),
    ]
)

model.summary()

W0506 14:22:01.650882  5196 deprecation.py:506] From C:\Users\YmeHweng\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 3

In [8]:
#compile and train the model
batch_size = 128
epochs = 8
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(data_train, label_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, verbose=2)

Train on 81000 samples, validate on 9000 samples
Epoch 1/8
81000/81000 - 21s - loss: 0.5678 - acc: 0.7097 - val_loss: 0.5063 - val_acc: 0.7554
Epoch 2/8
81000/81000 - 21s - loss: 0.5143 - acc: 0.7546 - val_loss: 0.4974 - val_acc: 0.7582
Epoch 3/8
81000/81000 - 21s - loss: 0.5084 - acc: 0.7583 - val_loss: 0.4940 - val_acc: 0.7634
Epoch 4/8
81000/81000 - 21s - loss: 0.5059 - acc: 0.7595 - val_loss: 0.4967 - val_acc: 0.7646
Epoch 5/8
81000/81000 - 21s - loss: 0.5067 - acc: 0.7595 - val_loss: 0.4928 - val_acc: 0.7644
Epoch 6/8
81000/81000 - 20s - loss: 0.5058 - acc: 0.7607 - val_loss: 0.4953 - val_acc: 0.7639
Epoch 7/8
81000/81000 - 21s - loss: 0.5042 - acc: 0.7610 - val_loss: 0.4921 - val_acc: 0.7644
Epoch 8/8
81000/81000 - 21s - loss: 0.5053 - acc: 0.7598 - val_loss: 0.4942 - val_acc: 0.7634


In [9]:
#save the weights
if not os.path.exists('FNNmodel'):
    os.mkdir('FNNmodel')
model.save_weights('./FNNmodel/simple.h5')

In [10]:
#load the pre-trained weights and test
model.load_weights('./FNNmodel/simple.h5')
#check the accuracy
test_pred = model.predict(data_test)
test_pred = np.where(test_pred>0.5, 1, 0)
acc = ((test_pred == label_test).sum()/2) / label_test.shape[0]

print('Accuracy on test set: {}'.format(acc))

Accuracy on test set: 0.7676
